# Competitive race

Before you start working with this notebook, remember to:

* Copy racers into `Competitive Designs`
* Remove all directories `comp_semifinal_*` if any exist
* Remove directory `comp_final` if it exists

You also need to create a file called `students.json`, with an entry for every student who might submit, in the following format:

```json
[
    {
        "first": "Somefirstname",
        "last": "Somelastname",
        "netid": "somenetid"
    },
    {
        "first": "Anotherfirstname",
        "last": "Anotherlastname",
        "netid": "anothernetid"
    }
]
```

You can think of the syntax here as a list of dict's. Note that (unlike in python), there must *not* be a comma after the last element of the list or after the last element of each dict.

Import modules and configure the notebook.

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import secrets
import json
import shutil
import subprocess
import ae353_drone
import importlib
importlib.reload(ae353_drone)

Create and print seed so it is possible to reproduce the results.

In [ ]:
seed = secrets.randbits(32)
print(seed)

Create simulator.

In [ ]:
simulator = ae353_drone.Simulator(display=False, seed=seed)

Load student roster (if desired).

In [ ]:
with open('students.json', 'r') as infile:
    students = json.load(infile)

def get_student(students, netid):
    for student in students:
        if student['netid'] == netid:
            return student
    return None

Define source directory with all designs.

In [ ]:
srcdir_designs = 'Competitive Designs'

Make sure all files in source directory have lower-case names.

In [ ]:
srcdir = srcdir_designs
for file in os.listdir(srcdir):
    os.rename(os.path.join(srcdir, file), os.path.join(srcdir, file.lower()))

Make sure all PNG files in source directory really are PNG files. This requires [ImageMagick](https://imagemagick.org).

In [ ]:
srcdir = srcdir_designs
template_image = 'question_mark.png'
for file in os.listdir(srcdir):
    if file.endswith('.png'):
        completed_process = subprocess.run([
                    'convert',
                    os.path.join(srcdir, file),
                    os.path.join(srcdir, file),
                ], capture_output=True)
        if completed_process.returncode != 0:
            print(f'   ** FAILED on {file} (returncode: {completed_process.returncode}), replacing with template')
            shutil.copyfile(template_image, os.path.join(srcdir, file))

Load drones from source directory, overriding the maximum allowable number.

In [ ]:
simulator.clear_drones()
failures = simulator.load_drones(srcdir_designs, no_max_num_drones=True)

List disqualified drones.

In [ ]:
print(f'DISQUALIFIED ({len(failures)}):\n')
for failure in failures:
    student = get_student(students, failure)
    if student is None:
        name = ''
    else:
        name = f'{student["first"]} {student["last"]}'
    print(f' {failure:10s} : {name}')

List qualified drones.

In [ ]:
print(f'QUALIFIED ({len(simulator.drones)}):\n')
for drone in simulator.drones:
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    print(f' {drone["name"]:10s} : {student["first"]} {student["last"]}')

Choose number of drones to race in each semifinal.

In [ ]:
num_drones_per_semifinal = 5

Create semifinal races.

In [ ]:
# Get list of qualified racers
qualified = [drone['name'] for drone in simulator.drones]

# Shuffle order of this list
simulator.rng.shuffle(qualified)

# Create each race
num_races = 0
while True:
    racers = qualified[-num_drones_per_semifinal:]
    qualified = qualified[:-num_drones_per_semifinal]
    
    srcdir = srcdir_designs
    dstdir = f'comp_semifinal_{num_races}'
    os.mkdir(dstdir)
    for racer in racers:
        shutil.copyfile(os.path.join(srcdir, f'{racer}.py'), os.path.join(dstdir, f'{racer}.py'))
        shutil.copyfile(os.path.join(srcdir, f'{racer}.png'), os.path.join(dstdir, f'{racer}.png'))
    
    num_races += 1
    if len(qualified) == 0:
        break

print(f'Created {num_races} semifinal races')

os.mkdir('comp_final')

In [ ]:
index_of_race = 0

## Semifinal races

Run **all** of the following blocks of code for each semifinal race.

Print index of current race.

In [ ]:
if (index_of_race + 1) > num_races:
   raise Exception('Stop! You have already run all the semifinal races.') 

print(f'Running semifinal race {index_of_race + 1} / {num_races}')

Ready...

In [ ]:
# Name of directory with racers
srcdir = f'comp_semifinal_{index_of_race}'

# Clear drones
simulator.clear_drones()

# Move rings
simulator.move_rings()

# Load drones
simulator.load_drones(srcdir)

# Reset
simulator.reset()

Steady...

In [ ]:
simulator.camera_contestview()

num_drones = len(simulator.drones)
num_columns = 3
num_rows = np.ceil(num_drones / num_columns).astype(int)
fig, axs = plt.subplots(num_rows, num_columns, figsize=(12, 4 * num_rows))
[ax.set_axis_off() for ax in axs.flatten()]
for ax, drone in zip(axs.flatten(), simulator.drones):
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    im = plt.imread(os.path.join(srcdir, f'{drone["name"]}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'{drone["name"]}\n{student["first"]} {student["last"]}', fontsize=24)
    ax.axis('equal')

fig.tight_layout(h_pad=5)

Go!

In [ ]:
start_time = time.time()
simulator.run(max_time=30.0, contestview=True)
print(f'real time elapsed: {time.time() - start_time}')

Find winner.

In [ ]:
winning_name = None
winning_time = np.inf
for drone in simulator.drones:
    if drone['finish_time'] is None:
        continue
    if drone['finish_time'] < winning_time:
        winning_name = drone['name']
        winning_time = drone['finish_time']

if winning_name is None:
    print(f'There was no winner (nobody finished).')
else:
    print(f'The winner was {winning_name} with time {winning_time:.2f} seconds')
    srcdir = f'comp_semifinal_{index_of_race}'
    dstdir = 'comp_final'
    shutil.copyfile(os.path.join(srcdir, f'{winning_name}.py'), os.path.join(dstdir, f'{winning_name}.py'))
    shutil.copyfile(os.path.join(srcdir, f'{winning_name}.png'), os.path.join(dstdir, f'{winning_name}.png'))
    student = get_student(students, winning_name)
    if student is None:
        raise Exception(f'could not find student for this drone name: {winning_name}')
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.set_axis_off()
    im = plt.imread(os.path.join(srcdir, f'{winning_name}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'WINNER ({winning_time} seconds)\n\n{winning_name}\n{student["first"]} {student["last"]}', fontsize=24)
    ax.axis('equal')

Increment index of race.

In [ ]:
index_of_race += 1
if index_of_race == num_races:
    print('STOP! YOU ARE DONE WITH THE SEMIFINALS')

## Final race

Run the following blocks of code **once** for the final race, after you have run all of the semifinal races.

Ready...

In [ ]:
# Name of directory with racers
srcdir = f'comp_final'

# Clear drones
simulator.clear_drones()

# Move rings
simulator.move_rings()

# Load drones
simulator.load_drones(srcdir)

# Reset
simulator.reset()

Steady...

In [ ]:
simulator.camera_contestview()

num_drones = len(simulator.drones)
num_columns = 3
num_rows = np.ceil(num_drones / num_columns).astype(int)
fig, axs = plt.subplots(num_rows, num_columns, figsize=(12, 4 * num_rows))
[ax.set_axis_off() for ax in axs.flatten()]
for ax, drone in zip(axs.flatten(), simulator.drones):
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    im = plt.imread(os.path.join(srcdir, f'{drone["name"]}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'{drone["name"]}\n{student["first"]} {student["last"]}', fontsize=24)
    ax.axis('equal')

fig.tight_layout(h_pad=5)

Go!

In [ ]:
start_time = time.time()
simulator.run(max_time=30.0, contestview=True)
print(f'real time elapsed: {time.time() - start_time}')

Find winner.

In [ ]:
winning_name = None
winning_time = np.inf
for drone in simulator.drones:
    if drone['finish_time'] is None:
        continue
    if drone['finish_time'] < winning_time:
        winning_name = drone['name']
        winning_time = drone['finish_time']

if winning_name is None:
    print(f'There was no winner (nobody finished).')
else:
    print(f'The winner was {winning_name} with time {winning_time:.2f} seconds')
    student = get_student(students, winning_name)
    if student is None:
        raise Exception(f'could not find student for this drone name: {winning_name}')
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.set_axis_off()
    im = plt.imread(os.path.join(srcdir, f'{winning_name}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'WINNER ({winning_time} seconds)\n\n{winning_name}\n{student["first"]} {student["last"]}', fontsize=24)
    ax.axis('equal')